In [ ]:
import numpy as np
#import functools
#from tqdm import tqdm
from typing import Tuple, Callable, Dict, Union
import numpy as np
#from skimage import filters, transform
#from typing import Tuple
from abc import ABC, abstractmethod
from typing import List, Iterable, Tuple
#import numpy as np
#import argparse
from sklearn.preprocessing import LabelEncoder
#from collections import OrderedDict
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import categorical_crossentropy,binary_crossentropy
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
#from fr_utils import *
#from inception_blocks_v2 import *
# import sys
# import glob
# import logging
# #from keras.models import Sequential, Model
# import random
# from skimage.io import imread
# from skimage import img_as_ubyte
# import warnings
# warnings.filterwarnings('ignore')


In [ ]:
class IterableDataset(ABC):
    @property
    def maxsize(self):
        raise NotImplementedError
    @property
    def genuine_per_user(self):
        raise NotImplementedError
    @property
    def skilled_per_user(self):
        raise NotImplementedError
    @property
    
    def simple_per_user(self):
        raise NotImplementedError
    @abstractmethod
    def get_user_list(self) -> List[int]:
        pass
    @abstractmethod
    def iter_genuine(self, user: int) -> Iterable[Tuple[np.ndarray, str]]:
        pass
    @abstractmethod
    def iter_simple_forgery(self, user: int) -> Iterable[Tuple[np.ndarray, str]]:
        pass

    @abstractmethod
    def iter_forgery(self, user: int) -> Iterable[Tuple[np.ndarray, str]]:
        pass
# tf.enable_eager_execution()
class CedarDataset(IterableDataset):
    """ Helper class to load the CEDAR dataset
    """
    def __init__(self, path):
        self.path = path
        self.users = list(range(1, 55+1))

    @property
    def genuine_per_user(self):
        return 24

    @property
    def skilled_per_user(self):
        return 24

    @property
    def simple_per_user(self):
        return 0

    @property
    def maxsize(self):
        return 730, 1042

    def get_user_list(self):
        return self.users

    def iter_genuine(self, user):
        """ Iterate over genuine signatures for the given user"""

        files = ['{}_{}_{}.png'.format('original', user, img) for img in range(1, 24 + 1)]
        for f in files:
            full_path = os.path.join(self.path, 'full_org', f)
            img = imread(full_path, as_gray=True)
            yield img_as_ubyte(img), f

    def iter_forgery(self, user):
        """ Iterate over skilled forgeries for the given user"""

        files = ['{}_{}_{}.png'.format('forgeries', user, img) for img in range(1, 24 + 1)]
        for f in files:
            full_path = os.path.join(self.path, 'full_forg', f)
            img = imread(full_path, as_gray=True)
            yield img_as_ubyte(img), f

    def iter_simple_forgery(self, user):
        yield from ()  # No simple forgeries


In [ ]:
#path="/home/anuja/Downloads/signature_dataset/signatures/"
#npzpath="/home/anuja/Downloads/signature_dataset/signatures/signature.npz"
img_height = 150
img_width = 220
featurewise_std_normalization = True
batch_size = 5    
epochs=20
seed=42
input_shape=(img_height, img_width,1)
lamb = 0.95
#checkpoint=#pth_file
model=CedarDataset
users=55
#checkpointer = ModelCheckpoint(filepath="weights.{epoch:02d}-{val_loss:.2f}.hdf5",verbose=1, save_best_only=False,save_weights_only=False,save_freq=1)
checkpointer = ModelCheckpoint(filepath="weights.hdf5",monitor='val_loss', mode='min',verbose=1, save_best_only=True,save_weights_only=False)
#earlystoppetrain_file="/home/anuja/Downloads/signature_dataset/signatures/train_signature.npz"
test_file="/home/anuja/Downloads/signature_dataset/signatures/test_signature.npz"
#r = EarlyStopping(monitor='val_loss', min_delta=0,patience=4, verbose=1, mode='auto')
lr=0.001
lr_decay_multiplier=0.1
lr_decay_times=0.1
momentum=0.9
weight_decay=1e-4
epochs=20
seed=42
loss_type="L2"
forg=True
test=False #if true then it will test after loading the model 
input_size=(150,220)
#lamb in {0.95,0.99,0.999}
#checkpoint=#pth_file
lamb = 0.95
#checkpoint=#pth_file
model=CedarDataset
users=55
#no_images=10
n_classes=55


In [ ]:
data=pd.read_csv('/home/anuja/Downloads/signature_dataset/signatures.csv')
#data=data.sample(frac=1).reset_index(drop=True)


In [ ]:
data=pd.read_csv('triplet_file_random.csv')
#data=data.sample(frac=1).reset_index(drop=True)


In [ ]:
data.head()

In [ ]:
input_shape

In [ ]:
K.set_image_data_format('channels_last')
def train_model(input_shape):
    model=Sequential()
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_1', strides=(1,1), input_shape= input_shape, 
                    kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_2', strides=(1,1),kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_3', strides=(1,1),kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_4', strides=(2,2),kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(96, (3, 3), activation='relu', name='conv1_5', strides=(2,2),kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(MaxPooling2D((3,3), strides=(2, 2)))
    #model.add(ZeroPadding2D((2, 2),))
    model.add(Conv2D(256, (5, 5), activation='relu', name='conv2_1', strides=(1, 1),padding="same",kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(MaxPooling2D((3,3), strides=(2, 2)))
    #model.add(Dropout(0.3))# added extra
    #model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(384, (3, 3), activation='relu', name='conv3_1', strides=(1, 1),padding="same", kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(Conv2D(384, (3, 3), activation='relu', name='conv3_2', strides=(1, 1),padding="same", kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    #model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='conv3_3', strides=(1, 1),padding="same", kernel_initializer='glorot_uniform'))   
    model.add(MaxPooling2D((3,3), strides=(2, 2)))
    #model.add(Dropout(0.3))# added extra
    model.add(Flatten(name='flatten'))
    model.add(Dense(128, kernel_regularizer=l2(0.0005), activation='relu', kernel_initializer='glorot_uniform'))
    model.add(BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    #model.add(Dropout(0.5))
    model.add(Dense(128, kernel_regularizer=l2(0.0005), activation='relu', kernel_initializer='glorot_uniform')) # softmax changed to relu
    #last_features=BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)
    return(model)

In [ ]:
input_image = Input(shape=input_shape)
input_positive = Input(shape=input_shape)
input_negative = Input(shape=input_shape)
Shared_DNN = train_model([150,220,1,])
encoded_image=Shared_DNN(input_image)
encoded_positive=Shared_DNN(input_positive)
encoded_negative=Shared_DNN(input_negative)

In [ ]:
from tensorflow.keras.layers import concatenate
merged_vector = concatenate([encoded_image, encoded_positive, encoded_negative], axis=-1, name='merged_layer')
model = Model(inputs=[input_image,input_positive, input_negative], outputs=merged_vector)
data.insert(2, "Y", 1) 


In [ ]:
Y_dummy = np.ones((int(len(data)*.9),1))
Y_val = np.ones((int(len(data)*.1),1))
Y_dummy.shape
def triplet_loss(y_true, y_pred, alpha = 0.2):  
    anchor, positive, negative = y_pred[:,0], y_pred[:,1], y_pred[:,2]
    print(anchor, positive, negative)
    pos_dist = tf.reduce_sum(tf.square(anchor - positive))
    neg_dist = tf.reduce_sum(tf.square(anchor - negative))
    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.maximum(basic_loss, 0)
    return loss

In [ ]:
Shared_DNN.summary()


In [ ]:
adam_optimizer = tf.keras.optimizers.Adam(lr=0.0001,amsgrad=True)
model.compile(loss=triplet_loss, optimizer=adam_optimizer)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
#train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,oom_range=0.2,horizontal_flip=True)
train_datagen = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")
def gen_flow_for_three_inputs(train_datagen,df1,df2,df3,y,a,b):
    train_anchor=train_datagen.flow_from_dataframe(dataframe=data[a:b],x_col=df1,y_col=y, color_mode='grayscale',batch_size=5,seed=42,shuffle=True,class_mode="other",target_size=input_size)
    train_positive=train_datagen.flow_from_dataframe(dataframe=data[a:b],x_col=df2,y_col=y, color_mode='grayscale',batch_size=5,seed=42,shuffle=True,class_mode="other",target_size=input_size)
    train_negative=train_datagen.flow_from_dataframe(dataframe=data[a:b],x_col=df3,y_col=y, color_mode='grayscale',batch_size=5,seed=42,shuffle=True,class_mode="other",target_size=input_size)
    while True:
            anchor = train_anchor.next()
            positive = train_positive.next()
            negative=train_negative.next()
            yield [anchor[0], positive[0],negative[0]], anchor[1]
input_gen=gen_flow_for_three_inputs(train_datagen,"Anchor","Positive","Negative","Y",0,len(Y_dummy))
test_gen=gen_flow_for_three_inputs(train_datagen,"Anchor","Positive","Negative","Y",len(Y_dummy),len(data))

In [ ]:
batch_size=32
model.fit_generator(generator=input_gen,validation_data=test_gen,validation_steps=len(Y_val)/batch_size,epochs=2,shuffle=True,steps_per_epoch=len(Y_dummy)/batch_size,verbose=1,callbacks=[checkpointer])
